In [102]:
import pandapower as pp
from math import acos,tan

In [103]:
#use 11kV and use cleveland
#assume inductance is zero/negligible, because close together
#need to find capacity
subnet = pp.create_empty_network()

V_nom_kV = 0.240*(3)**(1/2)
V_intermediate_kV=11
V_high_kV=132
buses= []

r_m_high=0.320
x_m_high=0.120
c_high=0.32
max_i_high=0.228

r_m_low= 1.600 
x_m_low=0.140
c_low=1.650
max_i_low=0.096 



#create the 9 buses, which are at the 0.415 voltage
for i in range(8):
    buses.append(pp.create_bus(subnet,vn_kv=V_nom_kV, name=f'Bus{i}'))

#high voltage bus for trafo
buses.append(pp.create_bus(subnet, vn_kv=V_intermediate_kV, name='Bus 8 (Trafo intermediate)'))

#create buses 9 to 11 at 11kv
for i in range(9,12):
    buses.append(pp.create_bus(subnet,vn_kv=V_intermediate_kV, name=f'Bus dummy trafto intermediate {i}'))

#create buses 12 to 14 at 11kv
for i in range(12,15):
    buses.append(pp.create_bus(subnet,vn_kv=V_nom_kV, name=f'Bus dummy trafto nom {i}'))
    
#create buses 15 and 16
buses.append(pp.create_bus(subnet,vn_kv=V_intermediate_kV, name='Bus Trafo from grid/11kV'))
buses.append(pp.create_bus(subnet,vn_kv=V_high_kV, name='Bus Trafo from grid/132kV'))

# bus 16 now becomes an external grid
pp.create_ext_grid(subnet,bus=buses[16],vm_pu=1,name='Swing bus(Grid Connection)')
 


    
    
    
#create transformers
#43 houses per trafo

admd=0.003
safety_factor=1.5 #give it a higher rate load
sn_mva =1.5 #rated apparent power

#the one branch
pp.create_transformer_from_parameters(net=subnet, hv_bus=buses[8], lv_bus=buses[0], sn_mva=sn_mva, vn_hv_kv=V_intermediate_kV,vn_lv_kv=V_nom_kV,vkr_percent=1,vk_percent=10,pfe_kw=sn_mva/100*1000,i0_percent=5) 
#the other 3
for i in range(9,12):
    pp.create_transformer_from_parameters(net=subnet, hv_bus=buses[i], lv_bus=buses[i+3], sn_mva=sn_mva, vn_hv_kv=V_intermediate_kV,vn_lv_kv=V_nom_kV,vkr_percent=1,vk_percent=10,pfe_kw=sn_mva/100*1000,i0_percent=5) 

#the big one
pp.create_transformer_from_parameters(net=subnet, hv_bus=buses[16], lv_bus=buses[15], sn_mva=sn_mva, vn_hv_kv=V_high_kV,vn_lv_kv=V_intermediate_kV,vkr_percent=1,vk_percent=10,pfe_kw=sn_mva/100*1000,i0_percent=5) 

    

    
    
    
    
    


#create the loads

loads=[]
power_factor=0.9

p_mw_per_house=0.003
q_mvar_per_house=p_mw_per_house*tan(acos(power_factor))
p_mw_PV_per_house=-0.0012
q_mvar_PV_per_house=p_mw_PV_per_house*tan(acos(power_factor))
number_of_houses_normal=6
number_of_houses_extra=7

for i in range(1,7):
    loads.append(pp.create_load(subnet,bus=buses[i], p_mw=p_mw_per_house*number_of_houses_normal, q_mvar=q_mvar_per_house*number_of_houses_normal,  name=f'Combined load at bus{i}'))
    loads.append(pp.create_load(subnet,bus=buses[i], p_mw=p_mw_PV_per_house*number_of_houses_normal, q_mvar=q_mvar_PV_per_house*number_of_houses_normal,  name=f'Combined PV at bus{i}'))
#For the 7 house bus
loads.append(pp.create_load(subnet,bus=buses[7], p_mw=p_mw_per_house*number_of_houses_extra, q_mvar=q_mvar_per_house*number_of_houses_extra,  name=f'Combined load at bus{i}'))
loads.append(pp.create_load(subnet,bus=buses[7], p_mw=p_mw_PV_per_house*number_of_houses_extra, q_mvar=q_mvar_PV_per_house*number_of_houses_extra,  name=f'Combined PV at bus{i}'))


#dummy loads
for i in range(12,15):
    loads.append(pp.create_load(subnet,bus=buses[i], p_mw=p_mw_per_house*43, q_mvar=q_mvar_per_house*43,  name=f'Combined dummy load at bus{i}'))
    loads.append(pp.create_load(subnet,bus=buses[i], p_mw=p_mw_PV_per_house*43, q_mvar=q_mvar_PV_per_house*43,  name=f'Combined dummy PV at bus{i}'))

#create lines for branch
line_lengths=[0.05+0.075,0.150,0.150,0.150]

#left branch
for i in range(0,4):
    #remember to update later the actual values for x and q
    pp.create_line_from_parameters(subnet,from_bus=buses[i],to_bus=buses[i+1],length_km=line_lengths[i], r_ohm_per_km =r_m_low, x_ohm_per_km = x_m_low, c_nf_per_km = c_low, max_i_ka=max_i_low)
#right branch 

#connection from 0 to 5
pp.create_line_from_parameters(subnet,from_bus=buses[0],to_bus=buses[5],length_km=line_lengths[0], r_ohm_per_km =r_m_low, x_ohm_per_km = x_m_low, c_nf_per_km = c_low, max_i_ka=max_i_low)

for i in range(5,7) :
    #remember to update later the actual values for x and q
    pp.create_line_from_parameters(subnet,from_bus=buses[i],to_bus=buses[i+1],length_km=line_lengths[1],r_ohm_per_km =r_m_low, x_ohm_per_km = x_m_low, c_nf_per_km = c_low, max_i_ka=max_i_low)

#create rest of lines
long_lines=[0.495,0.300,0.300,0.495]
for i in range(8,12):
    pp.create_line_from_parameters(subnet,from_bus=buses[15],to_bus=buses[i],length_km=long_lines[i-8], r_ohm_per_km =r_m_high, x_ohm_per_km = x_m_high, c_nf_per_km = c_high, max_i_ka=max_i_high)



pp.runpp(subnet)


In [104]:
subnet.bus

,name,vn_kv,type,zone,in_service
0,Bus0,0.415692,b,None,True
1,Bus1,0.415692,b,None,True
2,Bus2,0.415692,b,None,True
3,Bus3,0.415692,b,None,True
4,Bus4,0.415692,b,None,True
5,Bus5,0.415692,b,None,True
6,Bus6,0.415692,b,None,True
7,Bus7,0.415692,b,None,True
8,Bus 8 (Trafo intermediate),11.000000,b,None,True
9,Bus dummy trafto intermediate 9,11.000000,b,None,True


In [105]:
subnet.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service
0,None,None,0,1,0.125,1.60,0.14,1.65,0.0,0.096,1.0,1,None,True
1,None,None,1,2,0.150,1.60,0.14,1.65,0.0,0.096,1.0,1,None,True
2,None,None,2,3,0.150,1.60,0.14,1.65,0.0,0.096,1.0,1,None,True
3,None,None,3,4,0.150,1.60,0.14,1.65,0.0,0.096,1.0,1,None,True
4,None,None,0,5,0.125,1.60,0.14,1.65,0.0,0.096,1.0,1,None,True
5,None,None,5,6,0.150,1.60,0.14,1.65,0.0,0.096,1.0,1,None,True
6,None,None,6,7,0.150,1.60,0.14,1.65,0.0,0.096,1.0,1,None,True
7,None,None,15,8,0.445,0.32,0.12,0.32,0.0,0.228,1.0,1,None,True
8,None,None,15,9,0.319,0.32,0.12,0.32,0.0,0.228,1.0,1,None,True
9,None,None,15,10,0.445,0.32,0.12,0.32,0.0,0.228,1.0,1,None,True


In [106]:
subnet.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type
0,Combined load at bus1,1,0.0180,0.008718,0.0,0.0,NaN,1.0,True,wye
1,Combined PV at bus1,1,-0.0072,-0.003487,0.0,0.0,NaN,1.0,True,wye
2,Combined load at bus2,2,0.0180,0.008718,0.0,0.0,NaN,1.0,True,wye
3,Combined PV at bus2,2,-0.0072,-0.003487,0.0,0.0,NaN,1.0,True,wye
4,Combined load at bus3,3,0.0180,0.008718,0.0,0.0,NaN,1.0,True,wye
5,Combined PV at bus3,3,-0.0072,-0.003487,0.0,0.0,NaN,1.0,True,wye
6,Combined load at bus4,4,0.0180,0.008718,0.0,0.0,NaN,1.0,True,wye
7,Combined PV at bus4,4,-0.0072,-0.003487,0.0,0.0,NaN,1.0,True,wye
8,Combined load at bus5,5,0.0180,0.008718,0.0,0.0,NaN,1.0,True,wye
9,Combined PV at bus5,5,-0.0072,-0.003487,0.0,0.0,NaN,1.0,True,wye


In [107]:
subnet.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,0.959494,-1.695732,0.000000,0.000000
1,0.895256,-0.372193,0.010800,0.005231
2,0.836181,0.994666,0.010800,0.005231
3,0.796188,2.019247,0.010800,0.005231
4,0.776002,2.571246,0.010800,0.005231
5,0.911995,-0.667198,0.010800,0.005231
6,0.872442,0.261535,0.010800,0.005231
7,0.850968,0.797471,0.012600,0.006102
8,0.965191,-1.329177,0.000000,0.000000
9,0.965251,-1.330412,0.000000,0.000000


In [108]:
subnet.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.051556,0.021654,-0.047625,-0.021310,0.003931,0.000344,0.080944,0.080944,0.080944,0.959494,-1.695732,0.895256,-0.372193,84.316792
1,0.036825,0.016079,-0.034027,-0.015834,0.002798,0.000245,0.062339,0.062339,0.062339,0.895256,-0.372193,0.836181,0.994666,64.936015
2,0.023227,0.010604,-0.021932,-0.010490,0.001295,0.000113,0.042410,0.042410,0.042410,0.836181,0.994666,0.796188,2.019247,44.177306
3,0.011132,0.005260,-0.010800,-0.005231,0.000332,0.000029,0.021478,0.021478,0.021478,0.796188,2.019247,0.776002,2.571246,22.372521
4,0.038019,0.016898,-0.035843,-0.016708,0.002176,0.000190,0.060225,0.060225,0.060225,0.959494,-1.695732,0.911995,-0.667198,62.734223
5,0.025043,0.011477,-0.023776,-0.011366,0.001267,0.000111,0.041953,0.041953,0.041953,0.911995,-0.667198,0.872442,0.261535,43.700838
6,0.012976,0.006135,-0.012600,-0.006102,0.000376,0.000033,0.022850,0.022850,0.022850,0.872442,0.261535,0.850968,0.797471,23.801879
7,0.103575,0.107578,-0.103547,-0.107573,0.000028,0.000006,0.008119,0.008119,0.008119,0.965366,-1.334152,0.965191,-1.329177,3.561158
8,0.091376,0.106361,-0.091358,-0.106358,0.000018,0.000003,0.007624,0.007624,0.007624,0.965366,-1.334152,0.965251,-1.330412,3.343827
9,0.091382,0.106356,-0.091357,-0.106351,0.000025,0.000004,0.007624,0.007624,0.007624,0.965366,-1.334152,0.965206,-1.328935,3.343849


In [109]:
subnet.res_load

,p_mw,q_mvar
0,0.0180,0.008718
1,-0.0072,-0.003487
2,0.0180,0.008718
3,-0.0072,-0.003487
4,0.0180,0.008718
5,-0.0072,-0.003487
6,0.0180,0.008718
7,-0.0072,-0.003487
8,0.0180,0.008718
9,-0.0072,-0.003487


In [101]:
#dont run pv as generator, model as negaitve load with q=0, for balaeric islands model as load